In [1]:
import torch
import torchvision.models as models
import numpy as np
import cv2
import os
import re
from PIL import Image
import matplotlib.pyplot as plt

## 以下是适配MSTAR数据集使用的格式转化函数，有需要的可以自行下载MSTAR数据集进行使用

In [2]:
def parse_mstar_header(file_path):
    """解析 MSTAR 文件的 ASCII 头信息 (增强版)"""
    try:
        with open(file_path, 'rb') as f:
            # 增加读取范围到 4096 字节，防止 header 过长
            header_bytes = f.read(4096)
            # 使用 latin-1 解码，它比 ascii 更鲁棒，能处理特殊字符
            header_text = header_bytes.decode('latin-1', errors='ignore')

            # 使用 re.IGNORECASE 忽略大小写，并兼容更多空格情况
            rows_match = re.search(r'numberOfRows\s*[:=]\s*(\d+)', header_text, re.I)
            cols_match = re.search(r'numberOfColumns\s*[:=]\s*(\d+)', header_text, re.I)

            if rows_match and cols_match:
                rows = int(rows_match.group(1))
                cols = int(cols_match.group(1))
                return rows, cols, True

            # --- 保底方案：如果正则没搜到，尝试 MSTAR 常见的尺寸 ---
            # 很多 MSTAR 文件的二进制数据正好是文件末尾的 128*128*4 或 100*100*4
            file_size = os.path.getsize(file_path)
            for size in [128, 129, 100, 147, 148, 158]:
                if file_size >= size * size * 4:
                    # 这是一个危险的假设，但在 Header 损坏时很有用
                    # 我们暂时还是返回 False，看看正则优化后能不能解决
                    pass

            return None, None, False
    except Exception as e:
        print(f"解析出错: {e}")
        return None, None, False


def convert_mstar_to_png(src_path, dst_path):
    """读取单张 MSTAR 数据并保存为 PNG"""
    try:
        rows, cols, found = parse_mstar_header(src_path)
        if not found:
            return False, "Not a valid MSTAR header"

        with open(src_path, 'rb') as f:
            all_content = f.read()
            data_size = rows * cols * 4
            data_part = all_content[-data_size:]  # 取末尾二进制数据

            if len(data_part) < data_size:
                return False, "File too small"

            img_array = np.frombuffer(data_part, dtype='>f4').reshape(rows, cols)

            # 线性拉伸 (防止图片全黑或全白)
            img_min, img_max = img_array.min(), img_array.max()
            if img_max > img_min:
                img_norm = (img_array - img_min) / (img_max - img_min) * 255
                img_8bit = img_norm.astype(np.uint8)
            else:
                img_8bit = np.zeros((rows, cols), dtype=np.uint8)

            cv2.imwrite(dst_path, cv2.cvtColor(img_8bit, cv2.COLOR_GRAY2BGR))
            return True, f"{rows}x{cols}"
    except Exception as e:
        return False, str(e)

## 以下是实现将MSTAR数据集中的.000文件转换为PNG格式的代码

In [9]:
input_root = '../data/soc'
output_root = '../data/MSTAR_soc'

print(f"🚀 启动扫描: {os.path.abspath(input_root)}")

stats = {}
total_count = 0
fail_count = 0

# 获取文件夹下的所有内容，先看看第一层有什么
print(f"📂 根目录内容预览: {os.listdir(input_root)[:5]}")

for root, dirs, files in os.walk(input_root):
    for file in files:
        # 更加宽松的正则：匹配点号后面跟着数字的文件
        if re.search(r'\.\d+', file) and not file.endswith('.png'):
            src_file = os.path.join(root, file)

            # 自动构建目录
            rel_path = os.path.relpath(root, input_root)
            save_dir = os.path.join(output_root, rel_path)
            os.makedirs(save_dir, exist_ok=True)

            new_name = file.replace('.', '_') + ".png"
            dst_file = os.path.join(save_dir, new_name)

            # 核心调试点：尝试转换并获取错误
            success, msg = convert_mstar_to_png(src_file, dst_file)

            if success:
                total_count += 1
                category = os.path.basename(root)
                stats[category] = stats.get(category, 0) + 1
                if total_count % 100 == 0:
                    print(f"✅ 已成功转化 {total_count} 张...")
            else:
                fail_count += 1
                if fail_count <= 5:  # 只打印前5个错误，避免刷屏
                    print(f"❌ 转化失败 [{file}]: {msg}")

if total_count == 0 and fail_count == 0:
    print("⚠️ 未发现任何匹配的文件。请确认后缀是否为 .000 这种格式？")
    if len(files) > 0:
        print(f"当前扫描到的文件名示例: {files[0]}")
else:
    print(f"\n✨ 任务结束！成功: {total_count}, 失败: {fail_count}")

# 统计展示
if stats:
    print("类别分布:", stats)

🚀 启动扫描: /home/suxin/SCKansformer-main/data/soc
📂 根目录内容预览: ['val', 'train', 'rename.py']
✅ 已成功转化 100 张...
✅ 已成功转化 200 张...
✅ 已成功转化 300 张...
✅ 已成功转化 400 张...
✅ 已成功转化 500 张...
✅ 已成功转化 600 张...
✅ 已成功转化 700 张...
✅ 已成功转化 800 张...
✅ 已成功转化 900 张...
✅ 已成功转化 1000 张...
✅ 已成功转化 1100 张...
✅ 已成功转化 1200 张...
✅ 已成功转化 1300 张...
✅ 已成功转化 1400 张...
✅ 已成功转化 1500 张...
✅ 已成功转化 1600 张...
✅ 已成功转化 1700 张...
✅ 已成功转化 1800 张...
✅ 已成功转化 1900 张...
✅ 已成功转化 2000 张...
✅ 已成功转化 2100 张...
✅ 已成功转化 2200 张...
✅ 已成功转化 2300 张...
✅ 已成功转化 2400 张...
✅ 已成功转化 2500 张...
✅ 已成功转化 2600 张...
✅ 已成功转化 2700 张...
✅ 已成功转化 2800 张...
✅ 已成功转化 2900 张...
✅ 已成功转化 3000 张...
✅ 已成功转化 3100 张...
✅ 已成功转化 3200 张...
✅ 已成功转化 3300 张...
✅ 已成功转化 3400 张...
✅ 已成功转化 3500 张...
✅ 已成功转化 3600 张...
✅ 已成功转化 3700 张...
✅ 已成功转化 3800 张...
✅ 已成功转化 3900 张...
✅ 已成功转化 4000 张...
✅ 已成功转化 4100 张...
✅ 已成功转化 4200 张...
✅ 已成功转化 4300 张...
✅ 已成功转化 4400 张...
✅ 已成功转化 4500 张...
✅ 已成功转化 4600 张...
✅ 已成功转化 4700 张...
✅ 已成功转化 4800 张...
✅ 已成功转化 4900 张...
✅ 已成功转化 5000 张...
✅ 已成功转化 5100 张...

✨ 

# 以下是下载一些我们需要的预训练模型，有需要取消注释即可

In [4]:
# torch.hub.set_dir('/home/suxin/SCKansformer-main/model_pth')
# model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
# model = models.efficientnet_v2_m(
#     weights=models.EfficientNet_V2_M_Weights.IMAGENET1K_V1
# )
# model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

# model = models.resnet50(
#     weights=models.ResNet50_Weights.IMAGENET1K_V1
# )
# model = models.resnet101(
#     weights=models.ResNet101_Weights.IMAGENET1K_V1
# )